In [1]:
#попытка 71 (Использовать именно ее)

import os
import cv2
import numpy as np
import random
import csv

class VideoProcessor:
    def __init__(self):
        pass

    def remove_black_frames(self, frame, target_height=224, target_width=224):
        frame_sum = np.sum(frame, axis=2)
        y_idx, x_idx = np.where(frame_sum > 0)

        if len(y_idx) == 0 or len(x_idx) == 0:
            return np.zeros((target_height, target_width, 3), dtype=np.uint8)

        y_min, y_max = max(0, y_idx.min()), min(frame.shape[0], y_idx.max())
        x_min, x_max = max(0, x_idx.min()), min(frame.shape[1], x_idx.max())

        cropped_frame = frame[y_min:y_max, x_min:x_max]

        resized_frame = cv2.resize(cropped_frame, (target_width, target_height))

        return resized_frame

    def process_video(self, frames):
        processed_frames = [self.remove_black_frames(frame) for frame in frames]
        return processed_frames

    def augment_video(self, frames):
        augmented_frames = []
        for _ in range(5):
            augmented_frames.extend(self._apply_augmentation(frames))
        return augmented_frames

    def _apply_augmentation(self, frames):
        augmented_frames = []
        flip = random.random() > 0.5
        brightness_factor = random.uniform(0.5, 1.5)
        contrast_factor = random.uniform(0.5, 1.5)
        saturation_factor = random.uniform(0.5, 1.5)
        rotation_angle = random.uniform(-15, 15)

        for frame in frames:
            if flip:
                frame = np.fliplr(frame)
            frame = cv2.convertScaleAbs(frame, alpha=brightness_factor, beta=0)
            frame = cv2.convertScaleAbs(frame, alpha=contrast_factor, beta=0)
            hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            hsv_frame = cv2.convertScaleAbs(hsv_frame, alpha=saturation_factor, beta=0)
            frame = cv2.cvtColor(hsv_frame, cv2.COLOR_HSV2BGR)

            noise_std = random.uniform(0, 20)
            noise = np.zeros_like(frame)
            cv2.randn(noise, 0, noise_std)
            frame = cv2.add(frame, noise)

            center = (frame.shape[1] // 2, frame.shape[0] // 2)
            rotation_matrix = cv2.getRotationMatrix2D(center, rotation_angle, 1.0)
            frame = cv2.warpAffine(frame, rotation_matrix, (frame.shape[1], frame.shape[0]))

            augmented_frames.append(frame)
        return augmented_frames

    def save_video(self, frames, output_path):
        frame_height, frame_width, _ = frames[0].shape
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))
        for frame in frames:
            out.write(frame)
        out.release()

    def process_and_augment_video(self, input_path, output_folder):
        cap = cv2.VideoCapture(input_path)

        if not cap.isOpened():
            print("Не удалось открыть видео:", input_path)
            return

        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
        cap.release()

        processed_frames = self.process_video(frames)
        augmented_frames = self.augment_video(processed_frames)

        filename = os.path.basename(input_path).replace('.avi', '')
        filename_without_extension = os.path.splitext(filename)[0]

        for i, frame_batch in enumerate(np.array_split(augmented_frames, 5)):
            augmented_output_path = output_folder+ "/"+ f'{filename_without_extension}_augmented_{i}.mp4'
            self.save_video(frame_batch, augmented_output_path)

        print(f'Обработка и аугментация видео {filename} завершены.')

input_folder = 'papkahuyapka'
output_folder = 'papkahuyapka/new_dataset'

video_processor = VideoProcessor()

csv_file = 'processed_videos.csv'
if not os.path.exists(csv_file):
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Processed Video'])

processed_videos = set()
with open(csv_file, mode='r') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        processed_videos.add(row[0])

for filename in os.listdir(input_folder):
    if filename.endswith('.avi'):
        input_path = os.path.join(input_folder, filename)
        if filename not in processed_videos:
            input_path = os.path.join(input_folder, filename)
            video_processor.process_and_augment_video(input_path, output_folder)
            with open(csv_file, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([filename])
        else:
            print(f'File {filename} also obrabotan')


File 23__track_302.avi also obrabotan
File 26__track_476.avi also obrabotan
File 27__track_864.avi also obrabotan
